In [9]:
import re
import emoji
import pandas as pd
import numpy as np
from tqdm import tqdm
from underthesea import word_tokenize
from unidecode import unidecode

import os
from googleapiclient.discovery import build
import requests
from dotenv import load_dotenv

In [2]:
# --- Cấu hình đường dẫn ---
input_path = r"C:/Users/thean/Documents/Social_Network/Social_Network_Projects/Dataset/Project2C_youtube_data_cleaned.csv"
output_path = r"C:/Users/thean/Documents/Social_Network/Social_Network_Projects/Dataset/Project2C_youtube_data_cleaned.csv"

In [3]:
data = pd.read_csv(
    "C:/Users/thean/Documents/Social_Network/Social_Network_Projects/Dataset/P2_Youtube_VideosVI.csv",
    encoding="utf-8"
)
data.head()

,video_id,title,description,published_at,channel,category_id,tags,view_count,like_count,comment_count
0,DtXSB9qOjgM,Hướng Dẫn chậm 4 bước CHA CHA CHA cơ bản cho n...,💥💥💥ĐÂY LÀ LINK VIDEO RÁP ĐỘNG TÁC VỚI NHẠC: 💥 ...,2019-04-08T07:27:25Z,Zumba LeO,26,zumbafitness|dance|zumbaleo,10201605,58535.0,1499.0
1,sTnAFuyfY34,HƯỚNG DẪN CHẬM NHẢY KHIÊU VŨ BACHATA [ Hướng D...,Khiêu vũ bachata [ Hướng Dẫn Khiêu Vũ - Dân Vũ...,2024-09-01T10:30:19Z,Mão Phượng Dancesport,22,Dân Vũ|Khiêu Vũ|Hướng Dẫn Khiêu Vũ Dân Vũ Cho ...,46983,190.0,4.0
2,hs4i-SSN3Rw,Hướng dẫn đi vệ sinh đúng cách #funny #review ...,NaN,2025-07-26T08:37:26Z,Chuyện Chị Em,22,NaN,12448877,44724.0,154.0
3,DRw9PZ8tcG0,Kungfu china master 137/Hướng dẫn múa gậy #霞光腊...,NaN,2024-11-29T14:10:15Z,DẠY CÔN NHỊ KHÚC ONLINE,17,NaN,92902963,2527349.0,24977.0
4,PLhaTXhPm30,Hướng Dẫn Ăn Chân Gà Chanh Sao Cho Đúng Cách! ...,Hướng Dẫn Ăn Chân Gà Chanh Sao Cho Đúng Cách! ...,2025-02-15T04:00:25Z,TẠP HÓA ĐỒ ĂN VẶT,22,anvat|amthuc|anuong|mukbang|food|shorts,1434823,18264.0,34.0


# Data Preprocessing

In [4]:
# Chọn cột 'description' và loại NA 
text_data = data[['description', 'category_id']].dropna()
text_data.head()

# Danh sách stopwords tiếng Việt
vietnamese_stopwords = set("""
à ạ ả á à ă ằ ắ ẳ ẵ ặ â ầ ấ ẩ ẫ ậ là ở và của trong cho với về như thì được
là các một những khi này chúng tôi bạn anh em chị cô chú bác mọi người video
kênh đăng ký like share xem ủng hộ cảm ơn nhé ơi à ha haha haha clip shorts 
""".split())


In [5]:
# Hàm làm sạch từng mô tả
def clean_text(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    
    # Lowercase
    text = text.lower()
    
    # Loại bỏ emoji, URL, email, số điện thoại, hashtags, mentions
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'http\S+|www\S+', ' ', text)
    text = re.sub(r'\S+@\S+', ' ', text)
    text = re.sub(r'\d{9,}', ' ', text)
    text = re.sub(r'[@#]\w+', ' ', text)
    
    # Loại bỏ ký tự đặc biệt, chỉ giữ lại chữ và khoảng trắng
    text = re.sub(r'[^a-zA-ZÀ-ỹ\s]', ' ', text)
    
    # Tokenize tiếng Việt
    tokens = word_tokenize(text, format="text").split()
    
    # Loại bỏ stopwords và từ ngắn
    tokens = [t for t in tokens if t not in vietnamese_stopwords and len(t) > 1]
    
    # Loại bỏ trùng lặp liền kề (ví dụ ha ha ha)
    clean_tokens = []
    for i, tok in enumerate(tokens):
        if i == 0 or tok != tokens[i-1]:
            clean_tokens.append(tok)
    
    return " ".join(clean_tokens)

In [6]:
#  Áp dụng pipeline
tqdm.pandas()
text_data["clean_description"] = text_data["description"].progress_apply(clean_text)

# Loại bỏ mô tả quá ngắn (ít hơn 5 token)
text_data["token_count"] = text_data["clean_description"].apply(lambda x: len(x.split()))
text_data = text_data[text_data["token_count"] >= 5].reset_index(drop=True)
print(f" Sau khi lọc: còn {len(text_data)} mô tả hợp lệ.")

100%|██████████| 8002/8002 [01:13<00:00, 109.44it/s]


 Sau khi lọc: còn 7653 mô tả hợp lệ.


In [10]:
load_dotenv()

API_KEY = os.getenv("YOUTUBE_API_KEY")
url = "https://www.googleapis.com/youtube/v3/videoCategories"

params = {
    "part": "snippet",
    "regionCode": "VN",
    "key": API_KEY
}

response = requests.get(url, params=params)
data = response.json()

# Lấy category_id và category_name
categories = []
for item in data.get("items", []):
    category_id = item["id"]
    category_name = item["snippet"]["title"]
    categories.append((category_id, category_name))

for cid, cname in categories:
    print(cid, cname)

1 Film & Animation
2 Autos & Vehicles
10 Music
15 Pets & Animals
17 Sports
18 Short Movies
19 Travel & Events
20 Gaming
21 Videoblogging
22 People & Blogs
23 Comedy
24 Entertainment
25 News & Politics
26 Howto & Style
27 Education
28 Science & Technology
30 Movies
31 Anime/Animation
32 Action/Adventure
33 Classics
34 Comedy
35 Documentary
36 Drama
37 Family
38 Foreign
39 Horror
40 Sci-Fi/Fantasy
41 Thriller
42 Shorts
43 Shows
44 Trailers


In [12]:
# Chuyển list tuple thành dict {category_id: category_name}
category_dict = {int(cid): cname for cid, cname in categories}

# Thêm cột category_name dựa vào category_id
text_data["category_name"] = text_data["category_id"].map(category_dict)

In [13]:
text_data.head(3)

,description,category_id,clean_description,token_count,category_name
0,💥💥💥ĐÂY LÀ LINK VIDEO RÁP ĐỘNG TÁC VỚI NHẠC: 💥 ...,26,đây link ráp_động_tác nhạc lại hiện lên tuyển_...,104,Howto & Style
1,Khiêu vũ bachata [ Hướng Dẫn Khiêu Vũ - Dân Vũ...,22,khiêu vũ_bachata hướng_dẫn khiêu_vũ dân_vũ mới...,9,People & Blogs
2,Hướng Dẫn Ăn Chân Gà Chanh Sao Cho Đúng Cách! ...,22,hướng_dẫn ăn chân gà chanh sao đúng cách tuyển...,62,People & Blogs


In [14]:
# Lưu kết quả
text_data.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f" Đã lưu dữ liệu đã xử lý tại: {output_path}")


 Đã lưu dữ liệu đã xử lý tại: C:/Users/thean/Documents/Social_Network/Social_Network_Projects/Dataset/Project2C_youtube_data_cleaned.csv


In [15]:
# Đọc file CSV đã làm sạch
text_data = pd.read_csv(input_path)
print(f" Số dòng dữ liệu: {len(text_data)}")

 Số dòng dữ liệu: 7653
